In [1]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.models import load_model

norm_size1 = 200 
norm_size2 = 1210 #200*1210大小的图片
EPOCHS = 100 
INIT_LR = 1e-3
classnum = 2
batch_size = 2
epoch_num = 100 # dataset.repeat() 的参数 可以取100轮

In [2]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def read_img(img_name, lable):
    image = tf.io.read_file(img_name) #读取bmp图像并且进行解码
    image = tf.image.decode_bmp(image)
    # image = (image - tf.reduce_min(image))/(tf.reduce_max(image)-tf.reduce_min(image)) 归一化
    image = tf.image.per_image_standardization(image) #标准化
    return image,lable

In [6]:

def create_dataset(x):
    classes = {'good', 'bad'} #good bad
    img_list = []
    label_list = []
    cwd_list = ['/content/drive/MyDrive/Colab_Notebooks/ShiChuang/data/train/','/content/drive/MyDrive/Colab_Notebooks/ShiChuang/data/test/']
    #存放训练以及测试两类图像的文件夹，本地运行改为./data/train/和./data/test/
    cwd = cwd_list[x]   
    for index,name in enumerate(classes):
        class_path = cwd + name + '/'
        for img_name in os.listdir(class_path):
            img_path = class_path + img_name
            img_list.append(img_path)#把文件名做成一个列表
            label_list.append(index)   #标签做成列表
    img_names = tf.convert_to_tensor(img_list, dtype=tf.string)  
    lables = tf.convert_to_tensor(label_list, dtype=tf.float32)  # 将图片名list和lable的list转换成Tensor类型

    dataset = tf.data.Dataset.from_tensor_slices((img_names,lables))  # 创建dataset，传入的需要是tensor类型 dataset是含有所有文件名以及标签的tensor
    dataset = dataset.map(read_img)   # 传入read_img函数，将图片名转为像素 利用map函数把tensor的文件名标签转化为像素值与标签
    dataset = dataset.shuffle(buffer_size=3000).batch(batch_size).repeat(epoch_num) #将dataset打乱，设置一次获取batch_size条数据  
    
    return dataset

In [7]:

dataset = create_dataset(0)#0训练集 1测试集 


In [ ]:
# iterator = tf.compat.v1.data.make_one_shot_iterator(dataset)

In [9]:
checkpointer = ModelCheckpoint(filepath='weights_best_Deset_model.hdf5',monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
#保存结果最好的模型                              
reduce = ReduceLROnPlateau(monitor='val_accuracy', patience=10,verbose=1,factor=0.5,min_lr=1e-6)
#评价指标不上升时学习率下降                                                   
log_dir = os.path.join("/logs")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
#可视化

In [10]:
model = DenseNet121(weights=None,input_shape=(norm_size1, norm_size2, 3), classes=classnum) #传入输入形状 类别数
optimizer = Adam(lr=INIT_LR)#优化器
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
history = model.fit(dataset,batch_size = 2,epochs=100,verbose=1,callbacks=[tensorboard_callback],shuffle = True)#传入dataset无法设置验证集的比例
model.save('my_model_Desnet.h5')

Epoch 1/100


In [ ]:
loss_trend_graph_path = r"WW_loss.jpg"
acc_trend_graph_path = r"WW_acc.jpg"
import matplotlib.pyplot as plt

print("Now,we start drawing the loss and acc trends graph...")


# summarize history for accuracy
fig = plt.figure(1)
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Model accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.savefig(acc_trend_graph_path)
plt.close(1)


# summarize history for loss
fig = plt.figure(2)
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.savefig(loss_trend_graph_path)
plt.close(2)
print("We are done, everything seems OK...")
# #windows系统设置10关机
os.system("shutdown -s -t 10")